# BOOK RECOMMENDER

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import cluster, datasets
from sklearn.cluster import KMeans

import random
from IPython.display import Markdown, display 


In [2]:
gr_data = pd.read_csv('./2. Clean_df/gr_data_FOR_CLUSTERING.csv')
gr_data = gr_data.drop(['Unnamed: 0'],axis=1)
display(gr_data.head(3),gr_data.shape)

,title,series,author,description,language,pages,isbn,book_format,publish_year,first_publish_year,...,William C. Morris YA Debut Award Nominee,Women's Prize for Fiction Nominee,Women's Prize for Fiction Nominee for Longlist,World Fantasy Award Nominee for Best Novel,World Fantasy Award for Best Novel,Zilveren Griffel,الجائزة العالمية للرواية العربية (أي باف) / International Prize for Arabic Fiction (IPAF) Nominee,pages2,cluster,normalized_title
0,The Hunger Games,The Hunger Games,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,200-300,9.78044E+12,Hardcover,2008,2008,...,0,0,0,0,0,0,0,374.0,0,the hunger games
1,Harry Potter and the Order of the Phoenix,Harry Potter,J.K. Rowling,There is a door at the end of a silent corrido...,English,700-800,9.78044E+12,Paperback,2004,2003,...,0,0,0,0,0,0,0,870.0,0,harry potter and the order of the phoenix
2,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,English,200-300,1E+13,Paperback,2006,1960,...,0,0,0,0,0,0,0,324.0,0,to kill a mockingbird


(48540, 865)

## Functions

In [23]:
def find_most_similar_book(book_searched, book_search_norm, book_matches, gr_data):
   
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics.pairwise import cosine_similarity
    import unidecode
    
    # Get cluster label of searched book
    book_cluster = book_matches['cluster'].iloc[0]

    # Get all books in same cluster as searched book
    same_cluster_books = gr_data[gr_data['cluster'] == book_cluster]

    # Extract numerical features for same_cluster_books
    features = same_cluster_books.select_dtypes(include=np.number).drop(columns=['cluster'])
    features

    # Scale numerical features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    features_scaled

    # Find index of recommended book within same_cluster_books
    recommended_book_idx = same_cluster_books[same_cluster_books['title'].str.contains(book_searched)].index[0]

    # Compute cosine similarity between books in same cluster
    sims = cosine_similarity(features_scaled)

    # Get the index of the most similar book
    most_similar_book_idx = np.argsort(sims[recommended_book_idx])[::-1][1]

    # Get the most similar book from same_cluster_books
    recommended_book = same_cluster_books.iloc[most_similar_book_idx]

    return recommended_book

In [ ]:
def find_most_similar_book(book_matches, gr_data, book_searched):
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics.pairwise import cosine_similarity
    
    # Get cluster label of searched book
    book_cluster = book_matches['cluster'].iloc[0]
    
    # Get all books in same cluster as searched book
    same_cluster_books = gr_data[gr_data['cluster'] == book_cluster]
    
    # Extract numerical features for same_cluster_books
    features = same_cluster_books.select_dtypes(include=np.number).drop(columns=['cluster'])

    # Scale numerical features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Find index of recommended book within same_cluster_books
    recommended_book_idx = same_cluster_books[same_cluster_books['title'].str.contains(book_searched)].index[0]

    # Compute cosine similarity between books in same cluster
    sims = cosine_similarity(features_scaled)
    
    # Get the index of the most similar book
    most_similar_book_idx = np.argsort(sims[recommended_book_idx])[::-1][1]

    # Get the most similar book from same_cluster_books
    recommended_book = same_cluster_books.iloc[most_similar_book_idx]
    
    return recommended_book


In [ ]:
recommended_book

In [ ]:
# Return corrected name in the output
#book_searched = gr_data.loc[gr_data['normalized_title'].str.contains(recommended_book), 'title'].iloc[0]
book_searched = recommended_book['title']
book_searched


In [24]:
def book_recommender(book_searched, gr_data):
    import unidecode
    import random
    from IPython.display import display, Markdown
    
    """Treated book input"""
    
    # Normalize book_searched to lowercase and strip leading/trailing whitespace
    book_search_norm = unidecode.unidecode(book_searched.lower().strip())

    # Check if the book is in gr_data:
    book_matches = gr_data[gr_data['normalized_title'].str.contains(book_search_norm)]
    
    # If book searches IS IN gr_data
    if not book_matches.empty:
        
        # Find the most similar book from the same cluster
        # Nested function that uses cosine similarity
        recommended_book = find_most_similar_book(book_searched, book_search_norm, book_matches, gr_data)
        
        # Return corrected name in the output
        #book_searched = gr_data.loc[gr_data['normalized_title'].str.contains(book_search_norm), 'title'].iloc[0]

        #recommended_book = recommended_book_function['title']
        #book_searched
        
        # Display the recommendation message
        message = random.choice([
            f"Based on your selection of '**{book_searched}**', we recommend the *hot* book '**{recommended_book['title']}**' by **{recommended_book['author']}**.",
            f"Looks like you've got great taste in books! We recommend the *sizzling* read '**{recommended_book['title']}**' by **{recommended_book['author']}** based on your choice of '**{book_searched}**'.",
            f"You've hit the jackpot with your choice of '**{book_searched}**'! Our recommendation for you is the *hyped* book '**{recommended_book['title']}**' by **{recommended_book['author']}**.",
            f"Based on your selection of '**{book_searched}**', we think you'll love the *popular* book '**{recommended_book['title']}**' by **{recommended_book['author']}**'.",
            f"Based on your selection of '**{book_searched}**', we suggest the *trending* novel '**{recommended_book['title']}**' by **{recommended_book['author']}**.'",
            f"Based on your selection of '**{book_searched}**', you might also like the *bestselling* book '**{recommended_book['title']}**' by **{recommended_book['author']}**.'",
            f"Looks like you're a big fan of **{book_searched}**! Our recommendation algorithm thinks you'll enjoy the hilarious and heartwarming **{recommended_book['title']}** by **{recommended_book['author']}**. Get ready to laugh and cry!",
            f"Oh joy, another person searching for **{book_searched}**. Haven't we had enough of that book already? But since you insist, our recommendation algorithm suggests the timeless classic **{recommended_book['title']}** by **{recommended_book['author']}**. Just kidding, it's probably just more of the same.",
            f"Based on your selection of '**{book_searched}**', we recommend the *hot* book '**{recommended_book['title']}**' by **{recommended_book['author']}**."
        ])
        
        display(Markdown(message))
        
        # Display book description
        display(Markdown(f"**Description:** {recommended_book['description']}"))

        # Display Goodreads rating
        display(Markdown(f"**GoodReads users rating:** {recommended_book['rating']}"))


        # Display Goodreads rating
        display(Markdown(f"**GoodReads users rating:** {recommended_book['rating']}"))
        
        # URL link to search for recommended book on Goodreads
        url_title = '+'.join(recommended_book['title'].split())
        url = f"https://www.goodreads.com/search?q={url_title}"
        display(Markdown(f"Search for the recommended book on Goodreads: {url}"))
        
    # If book search is NOT IN gr_data
    else:
        display(Markdown(f"Sorry, we couldn't find any books matching '**{book_searched.capitalize()}**'. Please try another search term."))


In [25]:
book_searched = input("Enter the name of the book you are interested in: ")
book = book_recommender(book_searched, gr_data)

Enter the name of the book you are interested in: the hunger games


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# Display the recommendation message
        

In [ ]:
message = random.choice([
            f"Based on your selection of '**{book_searched.capitalize()}**', we recommend the *hot* book '**{recommended_book['title']}**' by **{recommended_book['author']}**.",
            f"Looks like you've got great taste in books! We recommend the *sizzling* read '**{recommended_book['title']}**' by **{recommended_book['author']}** based on your choice of '**{book_searched.capitalize()}**'.",
            f"You've hit the jackpot with your choice of '**{book_searched.capitalize()}**'! Our recommendation for you is the *hyped* book '**{recommended_book['title']}**' by **{recommended_book['author']}**.",
            f"Based on your selection of '**{book_searched.capitalize()}**', we think you'll love the *popular* book '**{recommended_book['title']}**' by **{recommended_book['author']}**'.",
            f"Based on your selection of '**{book_searched.capitalize()}**', we suggest the *trending* novel '**{recommended_book['title']}**' by **{recommended_book['author']}**.'",
            f"Based on your selection of '**{book_searched.capitalize()}**', you might also like the *bestselling* book '**{recommended_book['title']}**' by **{recommended_book['author']}**.'",
            f"Looks like you're a big fan of **{book_searched.capitalize()}**! Our recommendation algorithm thinks you'll enjoy the hilarious and heartwarming **{recommended_book['title']}** by **{recommended_book['author']}**. Get ready to laugh and cry!",
            f"Oh joy, another person searching for **{book_searched.capitalize()}**. Haven't we had enough of that book already? But since you insist, our recommendation algorithm suggests the timeless classic **{recommended_book['title']}** by **{recommended_book['author']}**. Just kidding, it's probably just more of the same.",
            f"Based on your selection of '**{book_searched.capitalize()}**', we recommend the *hot* book '**{recommended_book['title']}**' by **{recommended_book['author']}**."
        ])

In [ ]:
# Get rid of accents in user input
    book_searched = unidecode.unidecode(book_searched)
    
    # Normalize book_searched to lowercase and strip leading/trailing whitespace
    book_searched = book_searched.lower().strip()

    # Check if the book is in gr_data:
    book_matches = gr_data[gr_data['title'].str.lower().str.contains(book_searched)]

In [ ]:
gr_data[['title','author']]

In [ ]:
def book_recommender(book_searched, gr_data):
    # Normalize input book name
    book_search_norm = normalize_book_title(book_searched)

    # Find cluster of searched book
    book_matches = find_book_cluster(book_search_norm, gr_data)
    same_cluster_books = find_same_cluster_books(book_matches, gr_data)

    # Check if same_cluster_books is empty
    if same_cluster_books.empty:
        return "Sorry, no books in the same cluster were found. Try searching for another book."

    # Find the most similar book from the same cluster
    recommended_book = find_most_similar_book(book_matches, gr_data, book_search_norm)

    # Return corrected name in the output
    return recommended_book


In [ ]:
book_searched = input("Enter the name of the book you are interested in: ")
book = book_recommender(book_searched, gr_data)

In [ ]:
# Accents

from unidecode import unidecode

# Apply unidecode to the title and author columns
gr_data['title'] = gr_data['title'].apply(unidecode)
gr_data['author'] = gr_data['author'].apply(unidecode)

# Now you can use the unaccented title and author columns for searching
book_matches = gr_data[gr_data['title'].str.lower().str.strip() == book_searched.lower().strip()]
author_matches = gr_data[gr_data['author'].str.lower().str.strip() == book_searched.lower().strip()]
